In [ ]:
def outstanding_orders():
    
    trads = ib.trades()
    
    for order_entry in pending_stop_loss_orders:
        
        if(order_entry[4] == "SELL"):
            
            trad_num = -404
            
            for tr in trads:
                if(trads[tr].order.orderId == order_entry[2]):
                    trad_num = tr
                    break
            
            
            if(trad_num != -404):
                
                contract = Stock(order_entry[0], 'SMART', 'USD')
                ib.qualifyContracts(contract)
                data = ib.reqMktData(contract)
                ib.sleep(2)
                current_price = round(data.marketPrice(), 2)           
            
                # WONT THESE +10% activate every 5 minutes all day????
                
                # testing
                current_time = datetime.datetime.now()
                f.write("Attempting stock SL updating\n")
                f.write(str(current_time) + "\n\n") 
            
                # current price OVER average buy price
                if((current_price / order_entry[3]) > 1.5):
                    trads[trad_num].order.auxPrice = round((order_entry[3] * 1.32), 2)
                    trads[trad_num].order.lmtPrice = round((order_entry[3] * 1.30), 2)
                    return
                elif((current_price / order_entry[3]) > 1.3):
                    trads[trad_num].order.auxPrice = round((order_entry[3] * 1.12), 2)
                    trads[trad_num].order.lmtPrice = round((order_entry[3] * 1.10), 2)
                    return
                elif((current_price / order_entry[3]) > 1.12):
                    trads[trad_num].order.auxPrice = round((order_entry[3] * 1.06), 2)
                    trads[trad_num].order.lmtPrice = round((order_entry[3] * 1.04), 2)
                    # testing
                    current_time = datetime.datetime.now()
                    f.write("Stock has surpassed +10% - grabbing profit stop loss\n")
                    f.write(str(order_entry[0]) + " has its SL moved to: " + str(trads[trad_num].order.auxPrice))
                    f.write(str(current_time) + "\n\n") 
                    return
                else:
                    return
                
        


def order_status(trade, fill):
    
    # I should probably still check if the status is FILLED before moving forward with a stop order to sell them!!!
    # fill in the orderstatus part after I confirm the rest of this stuff works...
    
    for order_entry in pending_stop_loss_orders:
        if(order_entry[2] == fill.execution.orderId):
            
            contract = Stock(fill.contract.symbol, 'SMART', 'USD')
            # stop limit with limit at 1.5 percent below Marubozu "low" and stop at the "low" .... not sure i like this...
            stopOrder = StopLimitOrder('SELL', fill.execution.shares, round((order_entry[3] * 0.985), 2), order_entry[3])
            stopOrderTrade = ib.placeOrder(contract, stopOrder)
            
            # tester
            current_time = datetime.datetime.now()
            f.write("Trade Order: " + str(fill.execution.orderId) + " has been filled, creating stop loss...")
            f.write("Stop Limit (SELL) Execution Order ID: " + str(stopOrderTrade.order.orderId) + "\n")
            f.write("Stop Order (SELL) -> Stop: " + str(order_entry[3]) + "\n")
            f.write(str(current_time) + "\n\n")
            
            new_order = [fill.contract.symbol, fill.execution.shares, stopOrderTrade.order.orderId, round(fill.execution.avgPrice, 2), "SELL"]
            pending_stop_loss_orders.remove(order_entry)            
            pending_stop_loss_orders.append(new_order)
            break